In [1]:
!pip install pandas scikit-learn imbalanced-learn


In [23]:
from google.colab import drive
import zipfile
import pandas as pd
import os


drive.mount('/content/drive')


zip_filename = '/content/creditcard.csv.zip'


extracted_folder = '/content/extracted_files/'
os.makedirs(extracted_folder, exist_ok=True)

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)


extracted_files = os.listdir(extracted_folder)
print("Extracted files:", extracted_files)


csv_files = [f for f in extracted_files if f.endswith('.csv')]
if len(csv_files) == 0:
    raise FileNotFoundError("No CSV files found in the ZIP archive.")
if len(csv_files) > 1:
    print("Multiple CSV files found. Selecting the first one.")

csv_filename = csv_files[0]
data = pd.read_csv(os.path.join(extracted_folder, csv_filename))


print(data.head())
print(data.info())
print(data['Class'].value_counts())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Extracted files: ['creditcard.csv']
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -

In [24]:

X = data.drop('Class', axis=1)
y = data['Class']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


smote = SMOTE(random_state=42)
rus = RandomUnderSampler(random_state=42)
pipeline = Pipeline(steps=[('o', smote), ('u', rus)])

X_train_balanced, y_train_balanced = pipeline.fit_resample(X_train_scaled, y_train)


In [26]:

y_pred_log_reg = log_reg.predict(X_test_scaled)
y_pred_rf = rf.predict(X_test_scaled)


print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_log_reg))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_log_reg))


print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))


Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.97      0.99     85307
           1       0.05      0.93      0.10       136

    accuracy                           0.97     85443
   macro avg       0.53      0.95      0.54     85443
weighted avg       1.00      0.97      0.98     85443

Confusion Matrix:
[[82973  2334]
 [   10   126]]
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85307
           1       0.56      0.96      0.71       136

    accuracy                           1.00     85443
   macro avg       0.78      0.98      0.85     85443
weighted avg       1.00      1.00      1.00     85443

Confusion Matrix:
[[85205   102]
 [    5   131]]


In [27]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import shuffle


X, y = shuffle(X, y, random_state=42)
X_sample, _, y_sample, _ = train_test_split(X, y, test_size=0.1, random_state=42)


X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.3, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


smote = SMOTE(random_state=42)
rus = RandomUnderSampler(random_state=42)
pipeline = Pipeline(steps=[('o', smote), ('u', rus)])
X_train_balanced, y_train_balanced = pipeline.fit_resample(X_train_scaled, y_train)


log_reg = LogisticRegression(max_iter=100, solver='liblinear', n_jobs=-1)
log_reg.fit(X_train_balanced, y_train_balanced)
with open('logistic_regression_model.pkl', 'wb') as f:
    pickle.dump(log_reg, f)


rf = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X_train_balanced, y_train_balanced)
with open('random_forest_model.pkl', 'wb') as f:
    pickle.dump(rf, f)


with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1223: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(


In [14]:
pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [31]:
import streamlit as st
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler


with open('logistic_regression_model.pkl', 'rb') as f:
    log_reg = pickle.load(f)

with open('random_forest_model.pkl', 'rb') as f:
    rf = pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)


def predict_fraud(transaction_data):

    transaction_df = pd.DataFrame([transaction_data], columns=transaction_columns)
    transaction_scaled = scaler.transform(transaction_df)


    log_reg_pred = log_reg.predict(transaction_scaled)[0]
    rf_pred = rf.predict(transaction_scaled)[0]

    return log_reg_pred, rf_pred

st.title('Credit Card Fraud Detection')

with st.form(key='fraud_form'):

    amount = st.number_input('Amount', min_value=0.0)
    time = st.number_input('Time', min_value=0.0)


    submit_button = st.form_submit_button(label='Check Fraud')

    if submit_button:

        transaction_data = {
            'Amount': amount,
            'Time': time,

        }


        log_reg_pred, rf_pred = predict_fraud(transaction_data)


        if log_reg_pred == 1:
            st.write('Logistic Regression: Transaction is fraudulent.')
        else:
            st.write('Logistic Regression: Transaction is genuine.')

        if rf_pred == 1:
            st.write('Random Forest: Transaction is fraudulent.')
        else:
            st.write('Random Forest: Transaction is genuine.')


2024-08-30 10:14:01.370 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-30 10:14:01.372 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-30 10:14:01.374 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-30 10:14:01.377 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-30 10:14:01.379 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-30 10:14:01.380 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-30 10:14:01.382 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-30 10:14:01.385 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [17]:
!pip install ipywidgets


  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [28]:
import pickle
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Load the trained models and scaler
with open('logistic_regression_model.pkl', 'rb') as f:
    log_reg = pickle.load(f)

with open('random_forest_model.pkl', 'rb') as f:
    rf = pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# Define the feature columns based on your dataset
# Ensure this list matches the order used in training
transaction_columns = [
    'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15',
    'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Time'
]

# Create interactive widgets for all features
widgets_dict = {col: widgets.FloatText(value=0.0, description=col + ':') for col in transaction_columns}

# Button to trigger prediction
predict_button = widgets.Button(description='Check Fraud')

def predict_fraud(features):
    # Create a DataFrame with the same columns and order as during training
    transaction_data = pd.DataFrame([features], columns=transaction_columns)

    # Scale the data
    transaction_scaled = scaler.transform(transaction_data)

    # Make predictions
    log_reg_pred = log_reg.predict(transaction_scaled)[0]
    rf_pred = rf.predict(transaction_scaled)[0]

    # Convert predictions to readable format
    log_reg_result = 'Fraudulent' if log_reg_pred == 1 else 'Genuine'
    rf_result = 'Fraudulent' if rf_pred == 1 else 'Genuine'

    return log_reg_result, rf_result

def on_button_clicked(b):
    # Gather all feature values from the widgets
    features = {col: widgets_dict[col].value for col in transaction_columns}

    # Predict fraud
    log_reg_result, rf_result = predict_fraud(features)

    # Display results
    print(f'Logistic Regression: Transaction is {log_reg_result}.')
    print(f'Random Forest: Transaction is {rf_result}.')

# Link the button click event to the function
predict_button.on_click(on_button_clicked)

# Display widgets
display(*widgets_dict.values())
display(predict_button)


FloatText(value=0.0, description='V1:')

FloatText(value=0.0, description='V2:')

FloatText(value=0.0, description='V3:')

FloatText(value=0.0, description='V4:')

FloatText(value=0.0, description='V5:')

...

FloatText(value=0.0, description='V27:')

FloatText(value=0.0, description='V28:')

FloatText(value=0.0, description='Amount:')

FloatText(value=0.0, description='Time:')

Button(description='Check Fraud', style=ButtonStyle())